In [8]:
import numpy as np
from PIL import Image

# 1. 设定目标
TARGET_H = 297
TARGET_W = 227

# 2. 直接打开图片 (确保是 RGB 模式)
im = Image.open('dxp.png').convert('RGB')
img_arr = np.array(im)

# 获取当前尺寸
h, w = img_arr.shape[:2]

# --- 核心 Numpy 处理：高度 (Height) ---
if h < TARGET_H:
    diff = TARGET_H - h
    pad_top = diff // 2
    pad_bot = diff - pad_top
    # 高度不够：上下平滑填充 (reflect)
    img_arr = np.pad(img_arr, ((pad_top, pad_bot), (0, 0), (0, 0)), mode='reflect')
elif h > TARGET_H:
    # 高度多了：中心裁剪
    start = (h - TARGET_H) // 2
    img_arr = img_arr[start : start+TARGET_H, :, :]

# --- 核心 Numpy 处理：宽度 (Width) ---
h, w = img_arr.shape[:2] # 更新一下当前尺寸

if w < TARGET_W:
    diff = TARGET_W - w
    pad_l = diff // 2
    pad_r = diff - pad_l
    # 宽度不够：左右平滑填充 (reflect)
    img_arr = np.pad(img_arr, ((0, 0), (pad_l, pad_r), (0, 0)), mode='reflect')
elif w > TARGET_W:
    # 宽度多了：中心裁剪
    start = (w - TARGET_W) // 2
    img_arr = img_arr[:, start : start+TARGET_W, :]



# 3. 转回 PIL 并输出
result_img = Image.fromarray(img_arr)
result_img.save('output_pad.jpg')

# 打印结果并弹窗展示
print(f"处理完成！最终尺寸: {result_img.size} (目标: 297x227)")
result_img.show()

处理完成！最终尺寸: (227, 297) (目标: 297x227)


In [10]:
import numpy as np
from PIL import Image
import cv2  # 用于对 Numpy 数组进行插值缩放

# 1. 设定目标尺寸
TARGET_H = 297
TARGET_W = 227

# 2. 直接打开图片
im = Image.open('dxp.png').convert('RGB')
img_arr = np.array(im)

# --- 核心处理：插值缩放 (Interpolation) ---
# cv2.resize 接收参数顺序是 (宽, 高) !!! 这一点和 numpy 的 shape (高, 宽) 是反的
# interpolation=cv2.INTER_CUBIC (三次插值) 比默认的双线性插值更清晰，适合"扩充像素"
img_arr = cv2.resize(img_arr, (TARGET_W, TARGET_H), interpolation=cv2.INTER_CUBIC)



# 3. 转回 PIL 并输出
result_img = Image.fromarray(img_arr)
result_img.save('output_interp.jpg')

# 打印结果并弹窗展示
print(f"处理完成！最终尺寸: {result_img.size} (目标: 297x227)")
result_img.show()

处理完成！最终尺寸: (227, 297) (目标: 297x227)


In [11]:
# ==========================================
# 笔记主题：[修正版] Python (OpenCV) 插值法扩充像素
# 场景：将图片强制调整为 宽227 x 高297 (竖版)
# ==========================================

import numpy as np
from PIL import Image
import cv2  # pip install opencv-python

def resize_image_interpolation():
    # 1. 设定目标尺寸 (Target Size)
    # -------------------------------------------------
    # 【修正】：宽 227，高 297
    TARGET_W = 227  # 宽度 Width
    TARGET_H = 297  # 高度 Height

    # 2. 读取图片
    # -------------------------------------------------
    input_path = 'dxp.png'
    output_path = 'output_fixed.jpg'
    
    # 转为 RGB 防止通道错误
    img_pil = Image.open(input_path).convert('RGB')
    img_arr = np.array(img_pil)

    # 3. 核心步骤：插值缩放
    # -------------------------------------------------
    # cv2.resize 参数顺序是 (Width, Height) -> (227, 297)
    # 这里的顺序必须是 (宽, 高)，千万别写反
    # interpolation=cv2.INTER_CUBIC (三次插值，放大效果最好)
    img_resized = cv2.resize(img_arr, (TARGET_W, TARGET_H), interpolation=cv2.INTER_CUBIC)

    # 4. 保存与展示
    # -------------------------------------------------
    result_img = Image.fromarray(img_resized)
    result_img.save(output_path)
    
    # 验证输出
    # Numpy shape 是 (Height, Width, Channel) -> 应该输出 (297, 227, 3)
    print(f"Numpy Shape (H,W): {img_resized.shape}") 
    # PIL Size 是 (Width, Height) -> 应该输出 (227, 297)
    print(f"PIL Size (W,H):   {result_img.size}")
    
    result_img.show()

if __name__ == '__main__':
    resize_image_interpolation()

Numpy Shape (H,W): (297, 227, 3)
PIL Size (W,H):   (227, 297)


In [20]:
import os
import cv2      # 必须安装: pip install opencv-python
import numpy as np
from PIL import Image # 必须安装: pip install pillow

# ==========================================
# 用户配置区
# ==========================================
INPUT_FOLDER = 'images_input'   # 输入文件夹 (你需要提前建好，并放入图片)
OUTPUT_FOLDER = 'images_output' # 输出文件夹 (代码会自动创建)

# 目标尺寸 (像素)
TARGET_W = 227  # 宽度
TARGET_H = 297  # 高度
# ==========================================

def smart_resize(img_arr):
    """
    智能判断缩放算法：
    1. 如果 原图像素 < 目标像素 (需要放大): 用 INTER_CUBIC (细节更好)
    2. 如果 原图像素 > 目标像素 (需要缩小): 用 INTER_AREA (无锯齿)
    """
    curr_h, curr_w = img_arr.shape[:2]
    curr_pixels = curr_h * curr_w
    target_pixels = TARGET_H * TARGET_W
    
    # 判断使用哪种插值方式
    if curr_pixels > target_pixels:
        interpolation = cv2.INTER_AREA
    else:
        interpolation = cv2.INTER_CUBIC
        
    # 注意: cv2.resize 接受的尺寸参数是 (宽, 高)
    return cv2.resize(img_arr, (TARGET_W, TARGET_H), interpolation=interpolation)

def main():
    # 1. 安全检查：输入文件夹是否存在
    if not os.path.exists(INPUT_FOLDER):
        print(f"❌ 错误：找不到输入文件夹 '{INPUT_FOLDER}'")
        print(f"   -> 请在当前代码目录下新建一个名为 '{INPUT_FOLDER}' 的文件夹，并放入你的图片。")
        return

    # 2. 自动建立输出文件夹
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)
        print(f"✅ 已创建输出文件夹: {OUTPUT_FOLDER}")

    # 支持的图片后缀
    valid_exts = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
    
    # 获取所有文件
    files = os.listdir(INPUT_FOLDER)
    total_files = len(files)
    count = 0
    
    print(f"\n🚀 开始处理... (目标尺寸: 宽{TARGET_W} x 高{TARGET_H})")
    print("-" * 50)

    for filename in files:
        name, ext = os.path.splitext(filename)
        
        # 过滤掉非图片文件 (比如 .txt 或 .DS_Store)
        if ext.lower() not in valid_exts:
            continue
            
        # 构造路径
        input_path = os.path.join(INPUT_FOLDER, filename)
        
        # 构造新文件名: 原名 + _output + 原后缀
        new_filename = f"{name}_output{ext}"
        output_path = os.path.join(OUTPUT_FOLDER, new_filename)
        
        try:
            # Step 1: 读取 (PIL -> Numpy)
            # convert('RGB') 确保统一为3通道，防止PNG透明通道导致报错
            img_pil = Image.open(input_path).convert('RGB')
            img_arr = np.array(img_pil)
            
            # Step 2: 核心处理 (智能插值)
            res_arr = smart_resize(img_arr)
            
            # Step 3: 保存
            res_img = Image.fromarray(res_arr)
            res_img.save(output_path)
            
            count += 1
            print(f"[{count}] 处理成功: {new_filename}")
            
        except Exception as e:
            print(f"⚠️  处理失败: {filename} -> 原因: {e}")

    print("-" * 50)
    print(f"🎉 全部完成！共处理 {count} 张图片")
    print(f"📂 输出目录: {os.path.abspath(OUTPUT_FOLDER)}")

if __name__ == '__main__':
    main()


🚀 开始处理... (目标尺寸: 宽227 x 高297)
--------------------------------------------------
[1] 处理成功: cg_output.jpg
[2] 处理成功: cy_output.jpg
[3] 处理成功: djx_output.jpg
[4] 处理成功: dxp_output.png
[5] 处理成功: gg_output.jpg
[6] 处理成功: hgf_output.jpg
[7] 处理成功: hjt_output.jpg
[8] 处理成功: hl_output.jpg
[9] 处理成功: jq_output.jpg
[10] 处理成功: jzm_output.jpg
[11] 处理成功: lb_output.jpg
[12] 处理成功: lbc_output.jpg
[13] 处理成功: ldz_output.jpg
[14] 处理成功: lf_output.jpg
[15] 处理成功: lkq_output.jpg
[16] 处理成功: lrh_output.jpg
[17] 处理成功: lrq_output.jpg
[18] 处理成功: lsq_output.jpg
[19] 处理成功: lx_output.jpg
[20] 处理成功: lyl_output.jpg
[21] 处理成功: myx_output.jpg
[22] 处理成功: mzd_output.jpg
[23] 处理成功: nrz_output.jpg
[24] 处理成功: pdh_output.jpg
[25] 处理成功: qqb_output.jpg
[26] 处理成功: qxs_output.jpg
[27] 处理成功: sql_output.jpg
[28] 处理成功: sy_output.jpg
[29] 处理成功: szs_output.jpg
[30] 处理成功: whw_output.jpg
[31] 处理成功: wjb_output.jpg
[32] 处理成功: xjp_output.jpg
[33] 处理成功: xxq_output.jpg
[34] 处理成功: yjy_output.jpg
[35] 处理成功: ylp_output.jpg
[36] 处理成功: ywy_output.jpg


In [18]:
import os
import numpy as np
from PIL import Image
import random

# ================= 配置区域 =================
# 1. 文件夹路径
# 输入和输出都在同一个文件夹
WORK_FOLDER = 'images_output'
FINAL_FILENAME = 'final_collage_beige.jpg' # 最终大图的文件名

# 2. 屏幕与图片参数
SCREEN_W, SCREEN_H = 3180, 2080  # 屏幕宽、高
IMG_W, IMG_H = 227, 297          # 小图宽、高

# 3. 颜色设置
# 米色 (Beige) 的 RGB 值
BG_COLOR = (245, 245, 220) 
# ===========================================

def get_diagonal_indices(max_rows, max_cols, count_needed):
    """
    生成器：生成副对角线顺序的坐标 (row, col)
    顺序：(0,0) -> (0,1),(1,0) -> (0,2),(1,1),(2,0) ...
    """
    coords = []
    # k 是行列索引之和 (row + col)
    # k 的最大值是 (max_rows-1) + (max_cols-1)
    max_k = max_rows + max_cols - 2
    
    for k in range(max_k + 1):
        # 在每一层 k 中，遍历 row
        # 这里设定 r 从 0 增加到 k，意味着方向是 ↙ (右上 -> 左下)
        for r in range(k + 1):
            c = k - r
            
            # 边界检查：必须在网格范围内
            if r < max_rows and c < max_cols:
                coords.append((r, c))
                
                # 如果凑够了数量，马上停止，节省计算
                if len(coords) >= count_needed:
                    return coords
    return coords

def main():
    # --- 1. 准备画布 ---
    print(f"🎨 正在初始化 {SCREEN_W}x{SCREEN_H} 的米色画布...")
    # np.full 用于填充单一颜色，参数: (形状, 颜色元组, 数据类型)
    canvas = np.full((SCREEN_H, SCREEN_W, 3), BG_COLOR, dtype=np.uint8)

    # --- 2. 读取图片 ---
    if not os.path.exists(WORK_FOLDER):
        print(f"❌ 错误：找不到文件夹 {WORK_FOLDER}")
        return

    # 获取所有图片，但要【排除】我们要生成的那个结果图，防止套娃
    valid_exts = {'.jpg', '.png', '.jpeg'}
    files = []
    for f in os.listdir(WORK_FOLDER):
        name, ext = os.path.splitext(f)
        # 排除非图片 以及 最终结果图本身
        if ext.lower() in valid_exts and f != FINAL_FILENAME:
            files.append(f)
            
    if not files:
        print("⚠️ 文件夹里没有可用的图片！")
        return

    print(f"📂 找到 {len(files)} 张素材图片，正在读取...")

    img_list = []
    for f in files:
        path = os.path.join(WORK_FOLDER, f)
        try:
            # 打开并转RGB
            img = Image.open(path).convert('RGB')
            arr = np.array(img)
            
            # 再次校验尺寸 (防止混入未处理的图)
            if arr.shape[:2] == (IMG_H, IMG_W):
                img_list.append(arr)
            else:
                print(f"   - 跳过尺寸不符的图片: {f} {arr.shape[:2]}")
        except Exception as e:
            print(f"   - 读取失败: {f} ({e})")

    if not img_list:
        print("❌ 没有符合尺寸要求的图片，终止。")
        return

    # --- 3. 乱序 ---
    print("🔀 正在打乱图片顺序...")
    random.shuffle(img_list)

    # --- 4. 计算网格与坐标 ---
    max_cols = SCREEN_W // IMG_W  # 3180 // 227 = 14
    max_rows = SCREEN_H // IMG_H  # 2080 // 297 = 7
    total_imgs = len(img_list)
    
    print(f"📊 网格容量: {max_rows}行 x {max_cols}列 (共 {max_rows*max_cols} 格)")
    print(f"📸 实际图片: {total_imgs} 张 (将只填充左上角区域)")

    # 获取对应数量的坐标
    indices = get_diagonal_indices(max_rows, max_cols, total_imgs)

    # --- 5. 拼图 (NumPy切片赋值) ---
    print("🚀 开始绘制...")
    for i, (r, c) in enumerate(indices):
        # 获取当前图片数据
        current_img_data = img_list[i]
        
        # 计算像素位置
        y_start = r * IMG_H
        x_start = c * IMG_W
        y_end = y_start + IMG_H
        x_end = x_start + IMG_W
        
        # 赋值到画布
        canvas[y_start:y_end, x_start:x_end] = current_img_data

    # --- 6. 保存输出 ---
    save_path = os.path.join(WORK_FOLDER, FINAL_FILENAME)
    print(f"💾 正在保存到: {save_path} ...")
    
    final_img = Image.fromarray(canvas)
    final_img.save(save_path)
    
    print("-" * 40)
    print("✅ 处理完成！")
    print("米色背景的拼图已生成。")
    final_img.show()

if __name__ == '__main__':
    main()

🎨 正在初始化 3180x2080 的米色画布...
📂 找到 39 张素材图片，正在读取...
🔀 正在打乱图片顺序...
📊 网格容量: 7行 x 14列 (共 98 格)
📸 实际图片: 39 张 (将只填充左上角区域)
🚀 开始绘制...
💾 正在保存到: images_output\final_collage_beige.jpg ...
----------------------------------------
✅ 处理完成！
米色背景的拼图已生成。


In [19]:
import os
import numpy as np
from PIL import Image
import colorsys  # 用于 RGB 转 HSV

# ================= 配置区域 =================
WORK_FOLDER = 'images_output'
FINAL_FILENAME = 'gradient_collage.jpg' # 输出文件名

# 屏幕与图片参数
SCREEN_W, SCREEN_H = 3180, 2080
IMG_W, IMG_H = 227, 297

# 【背景颜色修改】
# 推荐：深炭灰色 (高雅，突出色彩)
BG_COLOR = (30, 30, 30) 
# 如果你想要之前的米色，改回: (245, 245, 220)
# ===========================================

def get_image_sort_key(img_arr):
    """
    计算图片的排序主键。
    逻辑：计算图片平均颜色 -> 转为 HSV -> 返回 H (色相)
    """
    # 1. 计算整张图片的平均 R, G, B 值 (降维)
    # axis=(0,1) 表示计算高度和宽度维度的平均值，留下通道维度
    mean_rgb = np.mean(img_arr, axis=(0, 1))
    
    # 2. 归一化到 0-1 范围 (colorsys 需要 0-1 输入)
    r, g, b = mean_rgb / 255.0
    
    # 3. 转换为 HSV
    h, s, v = colorsys.rgb_to_hsv(r, g, b)
    
    # 返回 h (色相) 作为排序依据
    # 如果你想按亮度排序 (从暗到亮)，可以改为 return v
    return h

def get_diagonal_indices(max_rows, max_cols, count_needed):
    """生成副对角线坐标 (同前文逻辑)"""
    coords = []
    max_k = max_rows + max_cols - 2
    for k in range(max_k + 1):
        for r in range(k + 1):
            c = k - r
            if r < max_rows and c < max_cols:
                coords.append((r, c))
                if len(coords) >= count_needed:
                    return coords
    return coords

def main():
    print(f"🎨 初始化画布 (背景色: {BG_COLOR})...")
    canvas = np.full((SCREEN_H, SCREEN_W, 3), BG_COLOR, dtype=np.uint8)

    # --- 1. 读取图片 ---
    if not os.path.exists(WORK_FOLDER):
        print("❌ 找不到文件夹")
        return

    valid_exts = {'.jpg', '.png', '.jpeg'}
    # 用于存储 (图片数据, 排序Key) 的列表
    img_data_list = [] 
    
    print("📂 正在读取图片并分析颜色...")
    
    # 遍历文件夹
    file_list = os.listdir(WORK_FOLDER)
    for f in file_list:
        name, ext = os.path.splitext(f)
        if ext.lower() in valid_exts and "collage" not in name: # 排除之前的拼图结果
            path = os.path.join(WORK_FOLDER, f)
            try:
                img = Image.open(path).convert('RGB')
                arr = np.array(img)
                if arr.shape[:2] == (IMG_H, IMG_W):
                    # 【核心】计算这张图的色相值(Hue)
                    sort_key = get_image_sort_key(arr)
                    # 将图片数据和它的颜色值打包存起来
                    img_data_list.append({
                        'arr': arr,
                        'key': sort_key,
                        'name': f
                    })
            except:
                pass

    if not img_data_list:
        print("❌ 没有可用的图片")
        return

    # --- 2. 排序 (核心步骤) ---
    print(f"📊 找到 {len(img_data_list)} 张图片，正在按【色相(Hue)】排序...")
    # 根据我们在字典里存的 'key' 进行排序
    img_data_list.sort(key=lambda x: x['key'])

    # --- 3. 获取坐标 ---
    max_cols = SCREEN_W // IMG_W
    max_rows = SCREEN_H // IMG_H
    indices = get_diagonal_indices(max_rows, max_cols, len(img_data_list))

    # --- 4. 绘图 ---
    print("🚀 开始按颜色渐变拼图...")
    for i, (r, c) in enumerate(indices):
        img_item = img_data_list[i]
        
        y = r * IMG_H
        x = c * IMG_W
        
        # 赋值
        canvas[y : y+IMG_H, x : x+IMG_W] = img_item['arr']

    # --- 5. 保存 ---
    save_path = os.path.join(WORK_FOLDER, FINAL_FILENAME)
    final_img = Image.fromarray(canvas)
    final_img.save(save_path)
    
    print("-" * 30)
    print(f"✅ 完成！已按光谱顺序排列。")
    print(f"输出文件: {save_path}")
    final_img.show()

if __name__ == '__main__':
    main()

🎨 初始化画布 (背景色: (30, 30, 30))...
📂 正在读取图片并分析颜色...
📊 找到 39 张图片，正在按【色相(Hue)】排序...
🚀 开始按颜色渐变拼图...
------------------------------
✅ 完成！已按光谱顺序排列。
输出文件: images_output\gradient_collage.jpg


In [21]:
import os
import numpy as np
from PIL import Image
import colorsys  # 用于颜色排序

# ================= 配置区域 =================
WORK_FOLDER = 'images_output'
FINAL_FILENAME = 'final_layout_with_mzd.jpg'

# 屏幕与图片参数
SCREEN_W, SCREEN_H = 3180, 2080
IMG_W, IMG_H = 227, 297

# 背景颜色：深炭灰 (推荐)
BG_COLOR = (30, 30, 30)

# 【关键设置】指定要放在左上角的文件名关键词
VIP_KEYWORD = 'mzd' 
# ===========================================

def get_image_hue(img_arr):
    """获取图片的色相 (Hue) 用于排序"""
    mean_rgb = np.mean(img_arr, axis=(0, 1))
    r, g, b = mean_rgb / 255.0
    h, s, v = colorsys.rgb_to_hsv(r, g, b)
    return h

def get_diagonal_indices(max_rows, max_cols, count_needed):
    """生成副对角线坐标 (0,0) -> (0,1),(1,0) -> ..."""
    coords = []
    max_k = max_rows + max_cols - 2
    for k in range(max_k + 1):
        for r in range(k + 1):
            c = k - r
            if r < max_rows and c < max_cols:
                coords.append((r, c))
                if len(coords) >= count_needed:
                    return coords
    return coords

def main():
    print(f"🎨 初始化画布...")
    canvas = np.full((SCREEN_H, SCREEN_W, 3), BG_COLOR, dtype=np.uint8)

    if not os.path.exists(WORK_FOLDER):
        print("❌ 错误：找不到文件夹")
        return

    # 容器
    vip_image = None      # 存放 mzd 图片
    normal_images = []    # 存放其他图片
    
    print("📂 正在读取并分类图片...")
    
    valid_exts = {'.jpg', '.png', '.jpeg'}
    files = os.listdir(WORK_FOLDER)
    
    for f in files:
        name, ext = os.path.splitext(f)
        # 排除非图片和以前生成的拼图
        if ext.lower() in valid_exts and "final" not in name:
            path = os.path.join(WORK_FOLDER, f)
            try:
                img = Image.open(path).convert('RGB')
                arr = np.array(img)
                
                # 尺寸校验
                if arr.shape[:2] == (IMG_H, IMG_W):
                    # 打包数据
                    item = {
                        'arr': arr,
                        'hue': get_image_hue(arr), # 顺便计算颜色
                        'name': f
                    }
                    
                    # --- 核心逻辑：分流 ---
                    # 如果文件名包含 'mzd' (不区分大小写)，则存入 VIP
                    if VIP_KEYWORD.lower() in name.lower():
                        vip_image = item
                        print(f"   ★ 发现置顶图片: {f}")
                    else:
                        normal_images.append(item)
                        
            except Exception as e:
                pass

    if not vip_image and not normal_images:
        print("❌ 没有可用图片。")
        return

    # --- 排序逻辑 ---
    print(f"📊 正在处理顺序...")
    
    # 1. 对普通图片按颜色 (Hue) 排序
    normal_images.sort(key=lambda x: x['hue'])
    
    # 2. 合并列表：VIP图片 + 排序后的普通图片
    final_list = []
    if vip_image:
        final_list.append(vip_image) # mzd 放第一个
    else:
        print(f"⚠️ 警告：未找到包含 '{VIP_KEYWORD}' 的图片，左上角将是普通图片。")
        
    final_list.extend(normal_images) # 追加剩下的

    # --- 绘图逻辑 ---
    max_cols = SCREEN_W // IMG_W
    max_rows = SCREEN_H // IMG_H
    
    # 获取坐标
    indices = get_diagonal_indices(max_rows, max_cols, len(final_list))

    print(f"🚀 开始绘制 (首张图片: {final_list[0]['name']})...")
    
    for i, (r, c) in enumerate(indices):
        img_data = final_list[i]['arr']
        
        y = r * IMG_H
        x = c * IMG_W
        
        canvas[y : y+IMG_H, x : x+IMG_W] = img_data

    # --- 保存 ---
    save_path = os.path.join(WORK_FOLDER, FINAL_FILENAME)
    Image.fromarray(canvas).save(save_path)
    print("-" * 30)
    print(f"✅ 完成！mzd 已置顶。")
    print(f"输出路径: {save_path}")
    
    # 展示
    Image.fromarray(canvas).show()

if __name__ == '__main__':
    main()

🎨 初始化画布...
📂 正在读取并分类图片...
   ★ 发现置顶图片: mzd_output.jpg
📊 正在处理顺序...
🚀 开始绘制 (首张图片: mzd_output.jpg)...
------------------------------
✅ 完成！mzd 已置顶。
输出路径: images_output\final_layout_with_mzd.jpg
